In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/tobigs/poetry_generater/FINAL/
import json
import os
import time
import copy
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import trange
from model_rnn_simple import *

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

# poetry_idx
vocab2id, id2vocab = construct_vocab(file_='/content/drive/My Drive/tobigs/poetry_generater/FINAL/data/vocab.korean_morp.list')
image_path = "/content/drive/My Drive/tobigs/poetry_generater/FINAL/data/kts/total/"
# vocab_path = 'datx`a/vocab_jamo.pkl'
batch_size = 128

# with open(vocab_path, 'rb') as f:
#     vocab = pickle.load(f)

max_poem_lens = 500
max_keyword_counts=120

transform = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))])

/content/drive/My Drive/tobigs/poetry_generater/FINAL


In [0]:
vocab = {"w2i" : vocab2id, "i2w" : id2vocab}

In [0]:
data_total = PoetryRNNData(image_path, transform, vocab,vocab2id, max_poem_lens, max_keyword_counts)

In [5]:
all_data = data_total.reconstruct()

100%|██████████| 10000/10000 [00:00<00:00, 46182.91it/s]


In [6]:
all_data.keys()

dict_keys(['beach', 'cave', 'island', 'lake', 'mountain', 'amusement park', 'palace', 'park', 'restaurant', 'tower'])

In [0]:
train, test, val = data_total.split(all_data)
train = [train[i]  for i in range(len(train)) if int(train[i]['keywords'][2]) != 0  ]
test = [test[i]  for i in range(len(test)) if int(test[i]['keywords'][2]) != 0  ]
val = [val[i]  for i in range(len(val)) if int(val[i]['keywords'][2]) != 0  ]

In [0]:
del val

In [8]:
len(train), len(test), len(val)

(8000, 1000, 1000)

In [75]:
test[0]

{'dec': [[2,
   1,
   3,
   2,
   1,
   1,
   19,
   3,
   2,
   2560,
   1,
   12,
   1,
   21,
   3,
   2,
   1,
   8,
   2560,
   1,
   3,
   2,
   1,
   12,
   2320,
   60,
   1,
   20,
   90,
   3654,
   3,
   2,
   1,
   20,
   1,
   12,
   3,
   2,
   2116,
   343,
   29,
   1,
   1,
   21,
   3,
   2,
   1,
   1118,
   8,
   1,
   16,
   3,
   2,
   1,
   11,
   2234,
   9,
   1,
   5490,
   3,
   2,
   1,
   4712,
   44,
   3,
   2,
   1,
   11,
   1,
   40,
   8,
   3417,
   3,
   2,
   1778,
   44,
   1,
   14,
   1,
   8,
   3,
   2,
   1,
   14,
   1,
   8,
   2687,
   3,
   2,
   203,
   20,
   450,
   1433,
   1,
   14,
   3244,
   21,
   3,
   2,
   8954,
   20,
   450,
   1433,
   1,
   14,
   2500,
   5490,
   3,
   2,
   859,
   22,
   78,
   2113,
   9,
   1,
   10,
   1,
   3,
   2,
   2113,
   1041,
   22,
   1,
   13,
   1,
   3,
   4],
  tensor(126, device='cuda:0'),
  tensor([   2,    1,    3,    2,    1,    1,   19,    3,    2, 2560,    1,   12,
             1

In [0]:
Train = PoetryRNNDataSplit(transform, vocab, train, max_poem_lens, max_keyword_counts)
Test = PoetryRNNDataSplit(transform, vocab, test, max_poem_lens, max_keyword_counts)
Val = PoetryRNNDataSplit(transform, vocab, val, max_poem_lens, max_keyword_counts)

In [0]:
Train

In [0]:
train_loader = DataLoader(Train, batch_size=64)

In [0]:
# train, dev = train_test_split(json_data, test_size=0.001, train_size=0.99, random_state=4)
# print("Length of train {} and length of dev {}".format(len(train), len(dev)))


prt_emb = torch.load("/content/drive/My Drive/tobigs/poetry_generater/FINAL/data/pytorch_model.bin")
weights = prt_emb['bert.embeddings.word_embeddings.weight']
embed_weights = torch.cat([weights[:5], weights[7:]], dim=0)

# embed_weights = torch.from_numpy(weights)
# embed_weights = torch.randn(249, 512)

max_topic_len = 20
max_length = 100
batch_size = 64
# train_data = prepare_data_loader(train, max_topic_len, max_length, batch_size, shuffle=True)
# dev_data = prepare_data_loader(dev, max_topic_len, max_length, batch_size=10, shuffle=False)

rnn_hidden_dim = 512
rnn_layer = 2
rnn_bidre = False  # setting it True seems to break the data structure
rnn_dropout = 0
dense_dim = 300
dense_h_dropout = 0.5

In [53]:
embed_weights.size(1)

768

In [0]:
import torchvision.models as models

In [15]:
model = PoetryRNN(encoder_embed=embed_weights, encoder_v_len=196,
                  encoder_k_len=120,
                  decoder_embed=embed_weights, rnn_hidden_dim=rnn_hidden_dim,
                  rnn_layers=rnn_layer,
                  rnn_bidre=rnn_bidre, rnn_dropout=rnn_dropout,
                  dense_dim=dense_dim, dense_h_dropout=dense_h_dropout,
                  freeze_embed=True)

model_check_point = "./model_results/fix_embed_model_check_point.pk"
if os.path.isfile(model_check_point):
    model.load_state_dict(torch.load(model_check_point, map_location='gpu'))
    model.train(True)
    print("Load previous training parameters.")

if use_cuda:
    model = model.cuda()
    print("Dump to cuda")

512
768
Dump to cuda


In [0]:
model.train(True)
import torch.nn.utils as torch_utils

learning_rate = 1e-3
loss_fn = nn.NLLLoss()
model_params = list(filter(lambda p: p.requires_grad, model.parameters()))
optimizer = optim.Adam(model_params, lr=learning_rate, amsgrad=True,weight_decay=0)
torch_utils.clip_grad_value_(model.parameters(), clip_value=0.25)
if os.path.isfile("check_point_optim.pkl"):
    optimizer.load_state_dict(torch.load("check_point_optim.pkl"))
    print("Load previous optimizer.")

lr_scheme = ReduceLROnPlateau(optimizer, mode='min', factor=0.4, patience=10, min_lr=1e-7,
                              verbose=True)

In [0]:
# load model
model = torch.load('rnn_model.pk')
model.load_state_dict(torch.load('model_check_point.pk', map_location='cpu'))
optimizer.load_state_dict(torch.load('optimizer_check_point.pk'))
all_loss = torch.load('all_loss_check_point.pk')


In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [55]:
counter = 0
# all_loss = []
t_range = trange(10)
for iteration in t_range:
    print("starting...")
    losses = []
    for batch in train_loader:
        out = sort_batches(batch)
        model.zero_grad()
        #print(out['x'])
        #print(out['keyword'])
        target_score, hidden = model(out['img'],
                                    # 
                                     out['keyword'][0], out['keyword'][1], out['keyword'][2],
                                    #  out['x_pre'][0], out['x_pre'][1], out['x_pre'][2],
                                     out['x'][0], out['x'][1], out['x'][2])        # batch*seq_len x vocab dim
        loss = loss_fn(target_score, out['y'])

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()
        # lr_scheme.step(loss.data[0])
        print("Current batch {}, loss {}".format(counter, loss.item()))
        losses.append(loss.item())
        counter += 1
        all_loss.append(loss.item())

    one_ite_loss = np.mean(losses)
    lr_scheme.step(one_ite_loss)
    print("One iteration loss {:.3f}".format(one_ite_loss))



  0%|          | 0/10 [00:00<?, ?it/s]

starting...
tensor([[   1,    1,  699,  ...,    0,    0,    0],
        [4314, 1018,    1,  ...,    0,    0,    0],
        [   1, 3255, 2218,  ...,    0,    0,    0],
        ...,
        [4314,  699, 3255,  ...,    0,    0,    0],
        [   1,    1, 1971,  ...,    0,    0,    0],
        [4314,    1, 2218,  ...,    0,    0,    0]], device='cuda:0')
before torch.Size([64, 512, 14, 14])
torch.Size([64, 196, 512])
tensor([[[-0.0470,  0.0217, -0.0154,  ..., -0.0391,  0.0253,  0.0215],
         [-0.0470,  0.0217, -0.0154,  ..., -0.0391,  0.0253,  0.0215],
         [ 0.0299,  0.1107, -0.0565,  ...,  0.0058,  0.0384, -0.0482],
         ...,
         [-0.0276,  0.0102, -0.0276,  ...,  0.0029, -0.0093, -0.0042],
         [-0.0276,  0.0102, -0.0276,  ...,  0.0029, -0.0093, -0.0042],
         [-0.0276,  0.0102, -0.0276,  ...,  0.0029, -0.0093, -0.0042]],

        [[-0.0825, -0.0106, -0.0255,  ...,  0.0079, -0.0494, -0.0242],
         [-0.0366,  0.0283, -0.0342,  ...,  0.0522, -0.0090, -0.0372

KeyboardInterrupt: ignored

In [0]:
torch.save(model.state_dict(), 'model_check_point.pk')
torch.save(model, 'rnn_model.pk')
torch.save(optimizer.state_dict(), 'optimizer_check_point.pk')
torch.save(all_loss, 'all_loss_check_point.pk')

import torch
a = torch.randn(10,5,2)
lengths = [5,4,3,2,1,1,2,3,4,5]
for i in range(len(lengths)):
    if lengths[i] < a[i,:].size(0):
        a[i, lengths[i]:, :] = 0

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type PoetryRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type VisualEncoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/pyt

<All keys matched successfully>

In [0]:
model = model.eval()

In [0]:
# optimizer.load_state_dict(torch.load('optimizer_check_point.pk'))
# model.load_state_dict(torch.load('all_loss_check_point.pk'))

### 테스트

In [56]:
model.train(False)

PoetryRNN(
  (encoder_v): VisualEncoder(
    (vgg19): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)

In [0]:
test_loader = DataLoader(Test, batch_size=10)

In [108]:
target = []
for i in range(20):
  batch = next(iter(test_loader))
  out = sort_batches(batch)
  target_score, hidden_state = model.predict(out['img'],
                                     out['keyword'][0], out['keyword'][1], out['keyword'][2],
                                     out['x'][0], out['x'][1], out['x'][2])
  model.zero_grad()
  target.append(target_score)

tensor([[4314,    1,  699,  ...,    0,    0,    0],
        [4314,    1,    1,  ...,    0,    0,    0],
        [   1, 3887,    1,  ...,    0,    0,    0],
        ...,
        [4314,    1, 3255,  ...,    0,    0,    0],
        [4314, 1530, 3255,  ...,    0,    0,    0],
        [4314,  699, 3255,  ...,    0,    0,    0]], device='cuda:0')
before torch.Size([10, 512, 14, 14])
torch.Size([10, 196, 512])
tensor([[[-0.0825, -0.0106, -0.0255,  ...,  0.0079, -0.0494, -0.0242],
         [-0.0470,  0.0217, -0.0154,  ..., -0.0391,  0.0253,  0.0215],
         [ 0.0299,  0.1107, -0.0565,  ...,  0.0058,  0.0384, -0.0482],
         ...,
         [-0.0276,  0.0102, -0.0276,  ...,  0.0029, -0.0093, -0.0042],
         [-0.0276,  0.0102, -0.0276,  ...,  0.0029, -0.0093, -0.0042],
         [-0.0276,  0.0102, -0.0276,  ...,  0.0029, -0.0093, -0.0042]],

        [[-0.0825, -0.0106, -0.0255,  ...,  0.0079, -0.0494, -0.0242],
         [-0.0470,  0.0217, -0.0154,  ..., -0.0391,  0.0253,  0.0215],
         

RuntimeError: ignored

In [109]:
len(target)

5

In [112]:
target_score = F.softmax(target[0], dim=-1)

RuntimeError: ignored

In [62]:
target[1].shape

torch.Size([1434, 30347])

In [0]:
pred = [ int(torch.argmax(target_score[i])) for i in range(len(target_score))]

In [0]:
id2vocab[i]

In [0]:
poem_list = [ id2vocab[i] for i in pred]

In [0]:
poem_list = ' '.join(poem_list).split('[STOP]')

In [81]:
poem_list[1]

' [UNK] [SEP] [CLS] [UNK] [UNK] 부터/JX_ [UNK] ㄴ/ETM_ 마음/NNG_ 에/JKB_ [SEP] [CLS] 가슴/NNG_ 설레/VV_ 며/EC_ 짐/NNG_ [UNK] [UNK] 챙기/VV_ 고서/EC_ [SEP] [CLS] [UNK] 하/XSA_ ㄴ/ETM_ 도시/NNG_ 와/JC_ 바쁘/VA_ ㄴ/ETM_ 일상/NNG_ 을/JKO_ 벗어나/VV_ 아/EC_ [SEP] [CLS] [UNK] [UNK] 유/NNG_ 롭/XSA_ 게/EC_ 휴가/NNG_ 떠나/VV_ 는/ETM_ 시기/NNG_ [SEP] [CLS] 즐겁/VA_ ㄴ/ETM_ 휴식/NNG_ [UNK] 하/XSV_ 여/EC_ 보/VX_ 는/ETM_ 여름/NNG_ [UNK] [SEP] [CLS] 맑/VA_ 은/ETM_ 물/NNG_ 이/JKS_ 소리/NNG_ 내/VV_ 며/EC_ [UNK] 는/ETM_ 계곡/NNG_ [SEP] [CLS] [UNK] 하/XSA_ ㄴ/ETM_ 바람/NNG_ 불/VV_ 어/EC_ 오/VV_ 는/ETM_ 숲/NNG_ 속/NNG_ [UNK] 이/MAG_ 든/EC_ [SEP] [CLS] 이르/VA_ ㄴ/ETM_ 아침/NNG_ [UNK] 끼/VV_ ㄴ/ETM_ [UNK] 의/JKG_ 풍경/NNG_ 들/XSN_ 이/JKS_ [SEP] [CLS] 한/MM_ 폭/NNG_ 의/JKG_ 그림/NNG_ 처럼/JKB_ [UNK] 는/ETM_ 오토/NNG_ [UNK] [SEP] [CLS] [UNK] ㄴ/ETM_ 바다/NNG_ 하얗/VA_ ㄴ/ETM_ [UNK] 펼쳐지/VV_ ㄴ/ETM_ [UNK] [SEP] [CLS] 가/VV_ 아/EC_ 보/VX_ ㄹ/ETM_ 만/NNB_ 하/XSA_ ㄴ/ETM_ 곳/NNG_ 으로/JKB_ 잘/MAG_ 알리/VV_ 어/EC_ 지/VX_ ㄴ/ETM_ [UNK] [UNK] 찾/VV_ 아/EC_ [SEP] [CLS] [UNK] ㄴ/ETM_ 사람/NNG_ 끼리/XSN_ 이/VCP_ 라면/EC_ 장소/NNG_ 야/JX_ 어디/NP_ 이/V

In [82]:
test[1]

{'dec': [[2,
   1,
   3,
   2,
   1,
   1,
   19,
   3,
   2,
   2560,
   1,
   12,
   1,
   21,
   3,
   2,
   1,
   8,
   2560,
   1,
   3,
   2,
   1,
   12,
   2320,
   60,
   1,
   20,
   90,
   3654,
   3,
   2,
   1,
   20,
   1,
   12,
   3,
   2,
   2116,
   343,
   29,
   1,
   1,
   21,
   3,
   2,
   1,
   1118,
   8,
   1,
   16,
   3,
   2,
   1,
   11,
   2234,
   9,
   1,
   5490,
   3,
   2,
   1,
   4712,
   44,
   3,
   2,
   1,
   11,
   1,
   40,
   8,
   3417,
   3,
   2,
   1778,
   44,
   1,
   14,
   1,
   8,
   3,
   2,
   1,
   14,
   1,
   8,
   2687,
   3,
   2,
   203,
   20,
   450,
   1433,
   1,
   14,
   3244,
   21,
   3,
   2,
   8954,
   20,
   450,
   1433,
   1,
   14,
   2500,
   5490,
   3,
   2,
   859,
   22,
   78,
   2113,
   9,
   1,
   10,
   1,
   3,
   2,
   2113,
   1041,
   22,
   1,
   13,
   1,
   3,
   4],
  tensor(126, device='cuda:0'),
  tensor([   2,    1,    3,    2,    1,    1,   19,    3,    2, 2560,    1,   12,
             1

In [78]:
b = ' '.join(a).split('[SEP]')
b

['[UNK] 가/VV_ 다가/EC_ [UNK] [UNK] 로/JKB_ 빠지/VV_ ㄴ다는/ETM_ 말/NNG_ 이/JKS_ ',
 ' [CLS] [UNK] 이/JKC_ 아니/VCN_ 라는/ETM_ 것/NNB_ ㄹ/JKO_ ',
 ' [CLS] [UNK] 포/NNG_ 의/JKG_ 매력/NNG_ 에/JKB_ 빠지/VV_ 어/EC_ 보/VX_ ㄴ/ETM_ 사람/NNG_ 은/JX_ 알/VV_ ㄴ다/EC_ ',
 ' [CLS] [UNK] 포/NNG_ 가/JKS_ 얼마나/MAG_ [UNK] ㄴ/ETM_ 곳/NNG_ 이/VCP_ ㄴ가/EC_ 를/JKO_ ',
 ' [CLS] [UNK] [UNK] 교/NNG_ 의/JKG_ [UNK] ㄴ/ETM_ 풍경/NNG_ 이야/JX_ 말/NNG_ 하/XSV_ ㄹ/ETM_ 것/NNB_ 도/JX_ 없/VA_ 고/EC_ ',
 ' [CLS] 호수/NNG_ 같/VA_ 은/ETM_ 바다/NNG_ 의/JKG_ 풍경/NNG_ 이야/JX_ 너무/MAG_ 도/JX_ [UNK] 하/XSA_ 고/EC_ ',
 ' [CLS] [UNK] 에/JKB_ 담/VV_ 은/ETM_ 바다/NNG_ 의/JKG_ [UNK] 맛/NNG_ 또한/MAG_ 천하/NNG_ 의/JKG_ [UNK] 이/VCP_ 지/EC_ ',
 ' [CLS] [UNK] 섬/NNG_ 의/JKG_ 매력/NNG_ 은/JX_ 더/MAG_ 하/XSV_ ㄹ/ETM_ [UNK] 도/JX_ 없/VA_ 고/EC_ ',
 ' [CLS] 여러/MM_ 섬/NNG_ 을/JKO_ 오가/VV_ 는/ETM_ 항구/NNG_ 의/JKG_ 배/NNG_ 들/XSN_ 이야/JX_ ',
 ' [CLS] 꿈/NNG_ 과/JC_ 행복/NNG_ 을/JKO_ 나르/VV_ 느라/EC_ [UNK] 이/JKS_ 없/VA_ 지/EC_ ',
 ' [CLS] 바다/NNG_ 에/JKB_ 는/JX_ [UNK] 이/JKS_ 검/VA_ 어/EC_ 지/VX_ 고/EC_ ',
 ' [CLS] [UNK] 들/XSN_ 도/JX_ 조용히/MAG_ 집/NNG_ 으로/JKB_